# 02 eval bar

**in**: tracking csv from 01_roboflow_track  
**out**: `data/out/eval_ts.csv`, `data/out/pass_df.csv`

formulas:
```
eval_raw = 0.45 * pc_diff + 0.35 * xt_diff + 0.20 * press_diff
eval_smooth = ema(eval_raw, alpha=0.35)
eval_bar = clip(100 * eval_smooth, -100, 100)
```

In [ ]:
# cell 1: load tracking csv

from pathlib import Path

import numpy as np
import pandas as pd
from scipy.spatial import Voronoi

TRACK_DIR = Path('../data/track')
OUT_DIR = Path('../data/out')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# pitch dimensions (meters) - matches roboflow config (12000x7000 cm)
PITCH_X = 120.0
PITCH_Y = 70.0

# find tracking csv
track_files = list(TRACK_DIR.glob('*_track.csv'))
if not track_files:
    raise FileNotFoundError(f'no tracking csv found in {TRACK_DIR}')

TRACK_CSV = track_files[0]
MATCH_NAME = TRACK_CSV.stem.replace('_track', '')
print(f'loading: {TRACK_CSV}')

track_df = pd.read_csv(TRACK_CSV)
print(f'shape: {track_df.shape}')
print(track_df.head())

In [ ]:
# cell 2: compute pitch control per frame (voronoi area)

def bounded_voronoi_area(points, team_ids, bounds=(0, PITCH_X, 0, PITCH_Y)):
    """
    Compute bounded Voronoi area for each team.
    Returns dict: {team_id: area_fraction}
    """
    xmin, xmax, ymin, ymax = bounds
    total_area = (xmax - xmin) * (ymax - ymin)
    
    if len(points) < 3:
        return {0: 0.5, 1: 0.5}
    
    # add mirror points for bounded voronoi
    pts = np.array(points)
    mirror_pts = []
    for p in pts:
        mirror_pts.append([2 * xmin - p[0], p[1]])  # left
        mirror_pts.append([2 * xmax - p[0], p[1]])  # right
        mirror_pts.append([p[0], 2 * ymin - p[1]])  # bottom
        mirror_pts.append([p[0], 2 * ymax - p[1]])  # top
    
    all_pts = np.vstack([pts, mirror_pts])
    
    try:
        vor = Voronoi(all_pts)
    except Exception:
        return {0: 0.5, 1: 0.5}
    
    # compute areas for original points only
    team_areas = {0: 0.0, 1: 0.0}
    n_orig = len(pts)
    
    for i in range(n_orig):
        region_idx = vor.point_region[i]
        if region_idx == -1:
            continue
        region = vor.regions[region_idx]
        if -1 in region or len(region) < 3:
            continue
        
        # clip polygon to bounds
        poly = vor.vertices[region]
        poly = np.clip(poly, [xmin, ymin], [xmax, ymax])
        
        # shoelace formula for area
        n = len(poly)
        area = 0.0
        for j in range(n):
            area += poly[j, 0] * poly[(j + 1) % n, 1]
            area -= poly[(j + 1) % n, 0] * poly[j, 1]
        area = abs(area) / 2.0
        
        tid = team_ids[i]
        if tid in team_areas:
            team_areas[tid] += area
    
    # normalize to fractions
    total = sum(team_areas.values())
    if total > 0:
        for t in team_areas:
            team_areas[t] /= total
    else:
        team_areas = {0: 0.5, 1: 0.5}
    
    return team_areas


def compute_pitch_control(frame_df):
    """
    Compute pitch control diff for a single frame.
    Returns pc_diff in [-1, 1]: positive = team 0 controls more
    """
    # filter to players only (team 0 and 1)
    players = frame_df[(frame_df['team'].isin([0, 1])) & (frame_df['cls'] == 'player')]
    
    if len(players) < 4:
        return 0.0
    
    points = players[['x', 'y']].values
    teams = players['team'].values
    
    areas = bounded_voronoi_area(points, teams)
    pc_diff = areas.get(0, 0.5) - areas.get(1, 0.5)
    return float(np.clip(pc_diff, -1, 1))


# compute pitch control for each frame
frames = track_df['frame'].unique()
pc_data = []

for f in frames:
    fdf = track_df[track_df['frame'] == f]
    pc_diff = compute_pitch_control(fdf)
    t_sec = fdf['t_sec'].iloc[0]
    pc_data.append({'frame': f, 't_sec': t_sec, 'pc_diff': pc_diff})

pc_df = pd.DataFrame(pc_data)
print(f'pitch control computed for {len(pc_df)} frames')
print(pc_df['pc_diff'].describe())

In [ ]:
# cell 3: compute xT per ball position

def xt_val(x, y):
    """
    Analytic xT proxy: higher near goal, centered on pitch.
    Returns value in [0, 1]
    """
    x_norm = np.clip(x / PITCH_X, 0.0, 1.0)
    y_center = np.exp(-((y - PITCH_Y / 2.0) ** 2) / (2.0 * 18.0**2))
    return (x_norm ** 1.8) * y_center


def get_ball_position(frame_df):
    """Get ball position from frame data."""
    ball = frame_df[frame_df['cls'] == 'ball']
    if len(ball) == 0:
        return None, None
    return float(ball['x'].iloc[0]), float(ball['y'].iloc[0])


def get_possession_team(frame_df, ball_x, ball_y, max_dist=10.0):
    """
    Determine which team has possession based on nearest player.
    Returns team_id (0 or 1) or None if no player nearby.
    """
    if ball_x is None:
        return None
    
    players = frame_df[(frame_df['team'].isin([0, 1])) & (frame_df['cls'] == 'player')]
    if len(players) == 0:
        return None
    
    dists = np.sqrt((players['x'] - ball_x)**2 + (players['y'] - ball_y)**2)
    min_idx = dists.idxmin()
    min_dist = dists[min_idx]
    
    if min_dist > max_dist:
        return None
    
    return int(players.loc[min_idx, 'team'])


# compute xT for each frame
xt_data = []
prev_xt = {0: 0.0, 1: 0.0}  # rolling xT by team
window = 120  # rolling window in seconds

for f in frames:
    fdf = track_df[track_df['frame'] == f]
    t_sec = fdf['t_sec'].iloc[0]
    
    ball_x, ball_y = get_ball_position(fdf)
    poss_team = get_possession_team(fdf, ball_x, ball_y)
    
    xt = 0.0
    if ball_x is not None:
        xt = xt_val(ball_x, ball_y)
    
    xt_data.append({
        'frame': f,
        't_sec': t_sec,
        'ball_x': ball_x,
        'ball_y': ball_y,
        'xt': xt,
        'poss_team': poss_team,
    })

xt_df = pd.DataFrame(xt_data)

# compute rolling xT diff (team 0 - team 1)
# use exponential weighting for smooth transitions
xt_df['xt_team0'] = xt_df.apply(
    lambda r: r['xt'] if r['poss_team'] == 0 else 0, axis=1
)
xt_df['xt_team1'] = xt_df.apply(
    lambda r: r['xt'] if r['poss_team'] == 1 else 0, axis=1
)

# rolling sum with ewm
span = 30  # frames
xt_df['xt0_roll'] = xt_df['xt_team0'].ewm(span=span).mean()
xt_df['xt1_roll'] = xt_df['xt_team1'].ewm(span=span).mean()
xt_df['xt_diff'] = np.tanh(5 * (xt_df['xt0_roll'] - xt_df['xt1_roll']))  # scale to [-1, 1]

print(f'xT computed for {len(xt_df)} frames')
print(xt_df[['xt', 'xt_diff']].describe())

In [ ]:
# cell 4: compute pressure (nearest defender dist)

def compute_pressure(frame_df, ball_x, ball_y, poss_team):
    """
    Compute pressure differential.
    Positive = team 0 is applying more pressure.
    """
    if ball_x is None or poss_team is None:
        return 0.0
    
    def_team = 1 - poss_team
    
    # defenders = opposing team players
    defenders = frame_df[(frame_df['team'] == def_team) & (frame_df['cls'] == 'player')]
    
    if len(defenders) == 0:
        return 0.0
    
    # compute nearest defender distance
    dists = np.sqrt((defenders['x'] - ball_x)**2 + (defenders['y'] - ball_y)**2)
    min_dist = dists.min()
    
    # pressure = inverse of distance, normalized
    # closer defender = more pressure on ball carrier
    pressure = np.clip(1.0 - min_dist / 15.0, 0, 1)  # 15m = no pressure
    
    # sign based on which team is pressing
    if poss_team == 1:  # team 0 is defending
        return pressure
    else:  # team 1 is defending
        return -pressure


# compute pressure for each frame
press_data = []

for i, row in xt_df.iterrows():
    f = row['frame']
    fdf = track_df[track_df['frame'] == f]
    
    press_diff = compute_pressure(
        fdf, row['ball_x'], row['ball_y'], row['poss_team']
    )
    press_data.append({'frame': f, 'press_diff': press_diff})

press_df = pd.DataFrame(press_data)
print(f'pressure computed for {len(press_df)} frames')
print(press_df['press_diff'].describe())

In [ ]:
# cell 5: eval bar formula + ema smoothing

# merge all components
eval_df = pc_df.merge(xt_df[['frame', 'xt_diff']], on='frame')
eval_df = eval_df.merge(press_df, on='frame')

# eval bar formula
W_PC = 0.45
W_XT = 0.35
W_PRESS = 0.20
ALPHA = 0.35

eval_df['eval_raw'] = (
    W_PC * eval_df['pc_diff'] +
    W_XT * eval_df['xt_diff'] +
    W_PRESS * eval_df['press_diff']
)

# EMA smoothing
eval_df['eval_smooth'] = eval_df['eval_raw'].ewm(alpha=ALPHA).mean()

# scale to [-100, 100]
eval_df['eval_bar'] = np.clip(100 * eval_df['eval_smooth'], -100, 100)

print('eval bar computed')
print(eval_df[['eval_raw', 'eval_smooth', 'eval_bar']].describe())

# plot eval bar over time
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 4))
ax.fill_between(eval_df['t_sec'], 0, eval_df['eval_bar'], 
                where=eval_df['eval_bar'] >= 0, alpha=0.7, color='#2196F3', label='Team 0')
ax.fill_between(eval_df['t_sec'], 0, eval_df['eval_bar'], 
                where=eval_df['eval_bar'] < 0, alpha=0.7, color='#f44336', label='Team 1')
ax.axhline(0, color='white', linewidth=1)
ax.set_xlim(eval_df['t_sec'].min(), eval_df['t_sec'].max())
ax.set_ylim(-100, 100)
ax.set_xlabel('time (sec)')
ax.set_ylabel('eval bar')
ax.set_title('Eval Bar Over Time')
ax.legend()
ax.set_facecolor('#1a1a1a')
fig.set_facecolor('#1a1a1a')
ax.tick_params(colors='white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.title.set_color('white')
plt.tight_layout()
plt.show()

In [ ]:
# cell 6: pass analytics

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-np.clip(z, -500, 500)))


def detect_passes(track_df, min_dist=5.0, max_dist=50.0):
    """
    Detect pass events from ball movement.
    Returns DataFrame with pass info.
    """
    ball_df = track_df[track_df['cls'] == 'ball'].copy()
    ball_df = ball_df.sort_values('frame').reset_index(drop=True)
    
    if len(ball_df) < 10:
        return pd.DataFrame()
    
    passes = []
    
    # compute ball velocity
    ball_df['dx'] = ball_df['x'].diff()
    ball_df['dy'] = ball_df['y'].diff()
    ball_df['dt'] = ball_df['t_sec'].diff()
    ball_df['speed'] = np.sqrt(ball_df['dx']**2 + ball_df['dy']**2) / ball_df['dt'].clip(0.01)
    
    # detect sudden speed increases (pass initiation)
    ball_df['speed_smooth'] = ball_df['speed'].rolling(5, min_periods=1).mean()
    ball_df['speed_jump'] = ball_df['speed'] - ball_df['speed_smooth'].shift(1)
    
    # threshold for pass detection
    speed_thresh = ball_df['speed'].quantile(0.8)
    
    in_pass = False
    pass_start = None
    
    for i, row in ball_df.iterrows():
        if not in_pass and row['speed'] > speed_thresh:
            in_pass = True
            pass_start = row
        elif in_pass and row['speed'] < speed_thresh * 0.5:
            in_pass = False
            if pass_start is not None:
                dist = np.sqrt((row['x'] - pass_start['x'])**2 + (row['y'] - pass_start['y'])**2)
                if min_dist < dist < max_dist:
                    passes.append({
                        'frame_start': int(pass_start['frame']),
                        'frame_end': int(row['frame']),
                        't_start': pass_start['t_sec'],
                        't_end': row['t_sec'],
                        'sx': pass_start['x'],
                        'sy': pass_start['y'],
                        'ex': row['x'],
                        'ey': row['y'],
                        'dist': dist,
                    })
    
    return pd.DataFrame(passes)


# detect passes
pass_df = detect_passes(track_df)
print(f'detected {len(pass_df)} passes')

if len(pass_df) > 0:
    # add pass features
    pass_df['angle_deg'] = np.degrees(np.arctan2(
        np.abs(pass_df['ey'] - pass_df['sy']),
        np.maximum(pass_df['ex'] - pass_df['sx'], 1e-6)
    ))
    
    # xT delta
    pass_df['xt_start'] = xt_val(pass_df['sx'].values, pass_df['sy'].values)
    pass_df['xt_end'] = xt_val(pass_df['ex'].values, pass_df['ey'].values)
    pass_df['delta_xt'] = pass_df['xt_end'] - pass_df['xt_start']
    
    # estimate pass success probability (simplified - no defender data in this pass)
    # use defaults for lane_gap, recv_space, def_cnt
    pass_df['lane_gap'] = 3.0  # default
    pass_df['recv_space'] = 2.5  # default
    pass_df['def_cnt'] = 1.5  # default
    
    z = (2.6 
         - 0.11 * pass_df['dist'] 
         + 0.35 * pass_df['lane_gap'] 
         + 0.22 * pass_df['recv_space'] 
         - 0.015 * pass_df['angle_deg'] 
         - 0.45 * pass_df['def_cnt'])
    pass_df['p_pass'] = sigmoid(z)
    
    # through-pass flag (forward pass > 16m)
    pass_df['is_through'] = ((pass_df['ex'] - pass_df['sx']) >= 16.0).astype(int)
    
    # safe vs creative tags
    pass_df['safe_tag'] = (pass_df['p_pass'] >= 0.72) & (pass_df['delta_xt'] < 0.03)
    pass_df['creative_tag'] = (pass_df['p_pass'] >= 0.45) & (pass_df['delta_xt'] >= 0.05)
    
    print(pass_df[['dist', 'p_pass', 'delta_xt', 'is_through', 'safe_tag', 'creative_tag']].describe())

In [ ]:
# cell 7: export csvs

# eval timeseries
eval_ts_csv = OUT_DIR / f'{MATCH_NAME}_eval_ts.csv'
eval_df.to_csv(eval_ts_csv, index=False)
print(f'saved: {eval_ts_csv}')

# pass dataframe
if len(pass_df) > 0:
    pass_csv = OUT_DIR / f'{MATCH_NAME}_pass_df.csv'
    pass_df.to_csv(pass_csv, index=False)
    print(f'saved: {pass_csv}')
else:
    print('no passes detected, skipping pass_df export')

print('\nexports complete')

## verification checklist

- [ ] eval_ts has values in [-100, 100]
- [ ] eval changes when possession changes
- [ ] pass_df has p_pass in [0, 1]
- [ ] through-pass flags look reasonable